# Heart Failure Prediction Notebook

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import scipy.stats as stats
from scipy.stats import kurtosis, skew
from scipy.stats import chi2_contingency
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
import umap
import sklearn.cluster as cluster
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import NearestNeighbors
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from lightgbm import LGBMClassifier

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/maialenberrondo/Documents/heart-failure/venv/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: /usr/local/opt/libomp/lib/libomp.dylib\n  Referenced from: /Users/maialenberrondo/Documents/heart-failure/venv/lib/python3.7/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/usr/local/opt/libomp/lib/libomp.dylib' (no such file), '/usr/local/lib/libomp.dylib' (no such file), '/usr/lib/libomp.dylib' (no such file)"]


## EDA

In [ ]:
data = pd.read_csv('../data/data.csv')

In [ ]:
data.head()

### Descriptive Analytics

In [ ]:
data.describe()

In [ ]:
data.shape

In [ ]:
data.dtypes

#### Univariate analisis

###### Numerical Features

In [ ]:
numerical_features = ['Age', 'RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak']
categorical_features = ['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'FastingBS', 'ST_Slope', 'HeartDisease']

In [ ]:
def plot_variable_distribution(variable_name): 
    plt.figure(figsize=(20, 10))
    sns.histplot(data=data, x=f"{variable_name}")
    plt.xlabel(f"{variable_name}")
    plt.savefig(f"../results/EDA/{variable_name}_distribution.png")
    plt.show()
    plt.close()

In [ ]:
for numerical_feature in numerical_features:
    plot_variable_distribution(numerical_feature)
    print( 'excess kurtosis of normal distribution (should be 0): {}'.format(kurtosis(data[f'{numerical_feature}'])))
    print( 'skewness of normal distribution (should be 0): {}'.format(skew(data[f'{numerical_feature}'])))

###### Categorical Features

In [ ]:
def plot_categorical_feature_distribution(variable_name):
    plt.figure(figsize=(20, 10))
    ax = sns.countplot(x=f"{variable_name}", data=data)
    for p in ax.patches:
        ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.2, p.get_height()), ha='center', va='top', color='white', size=18)
    plt.xlabel(f"{column}")
    plt.savefig(f"../results/EDA/{variable_name}_distribution.png")
    plt.show()
    plt.close()

In [ ]:
for column in categorical_features: 
    plot_categorical_feature_distribution(column)

We can conclude that the dataset is not imbalaced, since there is no minorty class and we have a almost 50 / 50 ratio between those how suffered a heart failure and those that no

In [ ]:
symetric_distribution_features = ['Age', 'Cholesterol', 'MaxHR']

In [ ]:
asymetric_distribution_features = ['RestingBP', 'Oldpeak']

### Missing Values

In [ ]:
msno.matrix(data)

In [ ]:
msno.bar(data)

There are no missing values in this dataset. But there are some anomalies in the cholesterol feature.

In [ ]:
number_of_zeroes = data['Cholesterol'][data['Cholesterol'] == 0].count()

In [ ]:
print(f'There are {number_of_zeroes} 0´s in Cholesterol')

### Outlier Detection

In [ ]:
def mean_3_std(variable_name):
    data_mean, data_std = np.mean(data[f'{variable_name}']), np.std(data[f'{variable_name}'])
    cut_off = data_std * 3
    lower, upper = data_mean - cut_off, data_mean + cut_off
    outliers_lower = [x for x in data[f'{variable_name}'] if x < lower]
    outliers_upper = [x for x in data[f'{variable_name}'] if x > upper]
    print(f'Variable {variable_name}: Lower outliers {outliers_lower}, Upper outliers {outliers_upper}, boundaries {lower}, {upper}')

In [ ]:
def plot_intercuartile_range(variable_name):
    plt.figure(figsize=(20, 10))
    sns.boxplot(data=data, x=f"{variable_name}")
    plt.xlabel(f"{variable_name}")
    plt.savefig(f"../results/EDA/{variable_name}_outliers.png")
    plt.show()
    plt.close()

In [ ]:
for symetric_distribution_feature in symetric_distribution_features:
    mean_3_std(symetric_distribution_feature)

In [ ]:
def iqr(variable_name):
    q75, q25 = np.percentile(data[f'{variable_name}'], [75 ,25])
    iqr = q75 - q25
    lower = q25 - iqr*1.5
    upper = q75 + iqr*1.5
    outliers_lower = [x for x in data[f'{variable_name}'] if x < lower]
    outliers_upper = [x for x in data[f'{variable_name}'] if x > upper]
    print(f'Variable {variable_name}: Lower outliers {outliers_lower}, {len(outliers_lower)}, Upper outliers {outliers_upper}, {len(outliers_upper)}, boundaries {lower}, {upper}')

In [ ]:
for asymetric_distribution_feature in asymetric_distribution_features:
    iqr(asymetric_distribution_feature)

In [ ]:
for numerical_feature in numerical_features:
    plot_intercuartile_range(numerical_feature)

### Multivariate Analysis

The target column in this case is very clear, HeartDisease. We already know that it is not balanced, but know we will try to detect any pattern. 

#### Numerical features: Plots

In [ ]:
def distribution_acording_to_target_value(variable_name):
    plt.figure(figsize=(20, 10))
    plt.title(f"Distribution of {variable_name}")
    ax = sns.violinplot(x="HeartDisease", y=f"{variable_name}",
                    data=data, palette="Set2", split=True,
                    scale="count")
    plt.xlabel(f"{variable_name}")
    plt.savefig(f"../results/EDA/violin_{variable_name}.png")
    plt.show()
    plt.close()
    
    plt.figure(figsize=(20, 10))
    ax = sns.boxplot(x="HeartDisease", y=f"{variable_name}", data=data, palette="Set2")
    plt.xlabel(f"{variable_name}")
    plt.savefig(f"../results/EDA/distribution_{variable_name}_target.png")
    plt.show()
    plt.close()

In [ ]:
for numerical_feature in numerical_features: 
    distribution_acording_to_target_value(numerical_feature)
    

#### Categorical features: Plots

In [ ]:
def categorical_distribution_acording_to_target_value(variable_name):
    plt.figure(figsize=(20, 10))
    plt.title(f"Distribution of {variable_name}")
    ax = sns.countplot(x=f"{variable_name}", hue="HeartDisease", data=data)
    for p in ax.patches:
        ax.annotate(f'\n{p.get_height()}', (p.get_x()+0.2, p.get_height()), ha='center', va='top', color='white', size=14)
    plt.xlabel(f"{variable_name}")
    plt.savefig(f"../results/EDA/distribution_{variable_name}_targetb.png")
    plt.show()
    plt.close()

In [ ]:
for categorical_feature in categorical_features[:-1]: 
    categorical_distribution_acording_to_target_value(categorical_feature)

#### ANOVA analysis

The idea of this analysis is to see if there is any connection between the target values and the different numerical variables.

In [ ]:
for numerical_feauture in numerical_features:
    none_relevant_features = []
    p_value = 0.05
    anova_test = stats.f_oneway(data[f'{numerical_feauture}'][data['HeartDisease'] == 0], data[f'{numerical_feauture}'][data['HeartDisease'] == 1])
    if anova_test[1] > p_value:
        none_relevant_features.append(feature)

#### Chi analysis

The idea of this analysis is to see if there is any connection between the target values and the different categorical variables.

In [ ]:
p_value = 0.05
for categorical_feature in categorical_features:
    ct = pd.crosstab(data[f'{categorical_feature}'], data['HeartDisease'], margins=True)
    ct = ct.drop("All", axis=1).drop("All", axis=0)
    obs = np.array(ct.values)
    if stats.chi2_contingency(obs)[0:3][1] > p_value:
        none_relevant_features.append(f'{categorical_feature}')
print(none_relevant_features) 

#### Decistion Tree: Feature understanding

The idea is to train a decision tree to understand what are the variables that affect most the target value, or if any variable does not affect it

In [ ]:
# Encoding:
decision_tree = data.copy()

object_cols = decision_tree.loc[:, data.dtypes == 'O']
object_type_columns = [col for col in object_cols.columns]

for column in object_type_columns:
    decision_tree[f'{column}'] = LabelEncoder().fit_transform(decision_tree[f'{column}'])
    
features_for_importance=decision_tree.columns.to_list()
features_for_importance.remove('HeartDisease')

In [ ]:
clasifier = DecisionTreeClassifier()
clasifier.fit(decision_tree.loc[:, decision_tree.columns != 'HeartDisease'], decision_tree['HeartDisease'])

In [ ]:
importances = clasifier.feature_importances_
indices = np.argsort(importances)

plt.figure(1)
plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), np.array(features_for_importance)[indices])
plt.xlabel('Relative Importance')

#### Correlations

In [ ]:
plt.figure(figsize=(20, 10))
matrix = np.triu(data[numerical_features].corr())
sns.heatmap(data[numerical_features].corr(), annot=True, mask=matrix)
plt.show()
plt.close()

In [ ]:
data.corr().unstack().sort_values().drop_duplicates().head(10)

In [ ]:
data.corr().unstack().sort_values().drop_duplicates().tail(10)

#### Unsupervised learning for hidden patterns: UMAP

In [ ]:
scaler = StandardScaler()

data_clustering = data.copy()

object_cols = data_clustering.loc[:, data_clustering.dtypes == 'O']
object_type_columns = [col for col in object_cols.columns]

for column in object_type_columns:
    data_clustering[f'{column}'] = LabelEncoder().fit_transform(data_clustering[f'{column}'])
data_clustering = scaler.fit_transform(data_clustering.loc[:, data.columns != 'HeartDisease'])

In [ ]:
umap_embeding = umap.UMAP(random_state=42).fit_transform(data_clustering)

In [ ]:
for n_of_components in range(0,12):
    pca = PCA(n_components=n_of_components)
    pca_embeding = pca.fit_transform(data_clustering)
    print(f'Number of Components :{n_of_components}, {pca.explained_variance_ratio_.sum()}')

In [ ]:
pca = PCA(n_components=2)
pca_embeding = pca.fit_transform(data_clustering)
print(pca.explained_variance_ratio_.sum())

In [ ]:
kmeans_true_labels = cluster.KMeans(n_clusters=2).fit_predict(data_clustering)
kmeans_umap_labels = cluster.KMeans(n_clusters=2).fit_predict(umap_embeding[:, 0].reshape(-1, 1), umap_embeding[:, 1].reshape(-1, 1))
kmeans_pca_labels = cluster.KMeans(n_clusters=2).fit_predict(pca_embeding[:, 0].reshape(-1, 1), pca_embeding[:, 1].reshape(-1, 1))

In [ ]:
def scatter_plot(axis1, axis2, color, title, xlabel, ylabel):
    plt.figure(figsize=(20,10))
    plt.scatter(axis1, axis2, c=color, cmap='Pastel1')
    plt.title(f'{title}')
    plt.xlabel(f'{xlabel}')
    plt.ylabel(f'{ylabel}')
    plt.show()
    plt.close()

In [ ]:
scatter_plot(axis1=pca_embeding[:, 0].reshape(-1, 1),
            axis2=pca_embeding[:, 1].reshape(-1, 1),
            color=data['HeartDisease'],
            title='Heart failure in PCA', 
            xlabel='Principal Component 1',
            ylabel='Principal Component 2')
scatter_plot(axis1=pca_embeding[:, 0].reshape(-1, 1),
            axis2=pca_embeding[:, 1].reshape(-1, 1),
            color=kmeans_true_labels,
            title='K-means represented in PCA',
            xlabel='Principal Component 1',
            ylabel='Principal Component 2')
scatter_plot(axis1=pca_embeding[:, 0].reshape(-1, 1),
            axis2=pca_embeding[:, 1].reshape(-1, 1),
            color=kmeans_pca_labels,
            title='K-means calculated with in PCA',
            xlabel='Principal Component 1',
            ylabel='Principal Component 2')
scatter_plot(axis1=pca_embeding[:, 0].reshape(-1, 1),
            axis2=pca_embeding[:, 1].reshape(-1, 1),
            color=kmeans_umap_labels,
            title='K-means calculated with in UMAP',
            xlabel='Principal Component 1',
            ylabel='Principal Component 2')

In [ ]:
scatter_plot(axis1=umap_embeding[:, 0].reshape(-1, 1),
            axis2=umap_embeding[:, 1].reshape(-1, 1),
            color=data['HeartDisease'],
            title='Heart failure in PCA', 
            xlabel='UMAP embedding 1',
            ylabel='UMAP embedding 2')
scatter_plot(axis1=umap_embeding[:, 0].reshape(-1, 1),
            axis2=umap_embeding[:, 1].reshape(-1, 1),
            color=kmeans_true_labels,
            title='K-means represented in PCA',
            xlabel='UMAP embedding 1',
            ylabel='UMAP embedding 2')
scatter_plot(axis1=umap_embeding[:, 0].reshape(-1, 1),
            axis2=umap_embeding[:, 1].reshape(-1, 1),
            color=kmeans_pca_labels,
            title='K-means calculated with in PCA',
            xlabel='UMAP embedding 1',
            ylabel='UMAP embedding 2')
scatter_plot(axis1=umap_embeding[:, 0].reshape(-1, 1),
            axis2=umap_embeding[:, 1].reshape(-1, 1),
            color=kmeans_umap_labels,
            title='K-means calculated with in UMAP',
            xlabel='UMAP embedding 1',
            ylabel='UMAP embedding 2')

In [ ]:
def get_results(label, method, method_name):
    """
    df = pd.DataFrame(list(zip(label, method)), columns =['Label', f'{method_name}'])
    print(df)
    print(df.groupby([f'{method_name}']).count())
    print(df.groupby([f'Label', f'{method_name}']))
    """
    print(method_name)
    cm = confusion_matrix(label, method)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot()
    plt.show()

In [ ]:
get_results(data['HeartDisease'], kmeans_true_labels, 'k means')
get_results(data['HeartDisease'], kmeans_pca_labels, 'k means PCA')
get_results(data['HeartDisease'], kmeans_umap_labels, 'k means UMAP')

### EDA: Conclusions

More than one dataset has been created, depending on different decissions that have been taken in different phases of the EDA. 


## Modeling

### Feature Engineering

In [ ]:
clf = GaussianNB()
clf = LogisticRegression(random_state=0)
clf = NearestNeighbors(n_neighbors=2)
clf = svm.SVC()

clf = RandomForestClassifier(max_depth=2, random_state=0)
clf = xgb.XGBClassifier()
clf = AdaBoostClassifier(n_estimators=100, random_state=0)
clf = GradientBoostingClassifier()
clf = LGBMClassifier()

# Neural Net - pytorch